# Subiectul 3 - impactul impartirii bazei de date
Completati codul de mai jos cu secventa necesara astfel incat sa afisati acuratetea modelului pe baza de date de **test**. Impartirea train-val-test se va face conform ponderii 40-30-30, iar antrenarea se va rula timp de 400 de epoci.

In [ ]:
import torch
import pandas as pd # utilitar folosit pentru gestionarea bazelor de date
from sklearn.model_selection import train_test_split # va fi utilizat pentru împărțirea bazei de date în subseturile dorite
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

RANDOM_SEED = 1 # pentru reproductibilitate
torch.manual_seed(RANDOM_SEED)

!gdown 1D-Ua952YzK95yPCJxzr8SWu7ZJGVROJd

In [ ]:
df = pd.read_csv('weatherAUS.csv')

# Baza de date conține o gamă largă de atribute, însă nu ne interesează toate.
# Facem o listă cu descriptorii care sunt de interes dintre toți cei disponibili
# TODO: testați cu diferiți descriptori pentru a vedea care dintre ei are o influență mai mare asupra capacității de predicție - adăugați cel puțin 2 alți descriptori existenți în baza de date
keep = ['MinTemp', 'MaxTemp', 'Rainfall', 'Humidity3pm', 'Pressure9am', 'RainToday', 'RainTomorrow']

# Din întreaga bază de date păstrăm doar datele asociate descriptorilor selectați mai sus
df_keep = df[keep]

# Unii dintre descriptori sunt reprezentați de valori logice binare (yes/no). Acestea trebuie interpretate sub forma unor valori pe care o rețea neuronală le poate procesa, adică valori numerice.
df_keep['RainToday'].replace({'No': 0, 'Yes': 1}, inplace = True)
df_keep['RainTomorrow'].replace({'No': 0, 'Yes': 1}, inplace = True)

# De asemenea, baza de date conține intrări pentru care nu sunt disponibili toți descriptorii (valori de NaN). O soluție trivială este să eliminăm complet aceste intrări.
# TODO: propuneți o altă strategie de abordare a datelor incomplete în acest context.
df_keep = df_keep.dropna(how='any')

rain = df_keep[df_keep['RainTomorrow']==1] # selectăm doar intrările pentru care a fost înregistrată ploaie
no_rain = df_keep[df_keep['RainTomorrow']==0] # selectăm doar intrările pentru care nu a fost înregistrată ploaie
no_rain = no_rain.sample(n=len(rain)) # alegem len(rain) eșantioane aleatoare din baza de date no_rain
df_keep = pd.concat([rain,no_rain],axis=0) # concatenăm cele 2 subseturi de date, formând o bază de date cu gradul dorit de dezechilibru

x = df_keep[keep[:-1]]
y = df_keep[keep[-1]]

In [ ]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

x_train_val, x_test, y_train_val, y_test = train_test_split(x, y, test_size=test_ratio)
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=val_ratio/(val_ratio + train_ratio))

x_train = torch.from_numpy(x_train.to_numpy()).float()
y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())

x_val = torch.from_numpy(x_val.to_numpy()).float()
y_val = torch.squeeze(torch.from_numpy(y_val.to_numpy()).float())

x_test = torch.from_numpy(x_test.to_numpy()).float()
y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

In [ ]:
class Net(torch.nn.Module):

  def __init__(self, n_features):
    super(Net, self).__init__()
    self.fc1 = torch.nn.Linear(n_features, 5) 
    self.fc2 = torch.nn.Linear(5, 3) 
    self.fc3 = torch.nn.Linear(3, 1) 

  def forward(self, x):
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.relu(self.fc2(x))
    return torch.sigmoid(self.fc3(x))

In [ ]:
net=Net(x_train.shape[1])
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x_train = x_train.to(device)
y_train = y_train.to(device)

x_val = x_val.to(device)
y_val = y_val.to(device)

x_test = x_test.to(device)
y_test = y_test.to(device)

net = net.to(device)

criterion = criterion.to(device)

Funcții auxiliare

In [ ]:
# Funcție de calcul a acurateții.
def calculate_accuracy(y_true, y_pred):
  predicted = y_pred.ge(.5).view(-1)
  return (y_true == predicted).sum().float() / len(y_true)

# Funcție de rotunjire a unui tensor la un anumit număr de zecimale 
def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)

Antrenarea propriu-zisă

In [ ]:
for epoch in range(1000):
    
    y_pred = net(x_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    
    if epoch % 100 == 0:
      train_acc = calculate_accuracy(y_train, y_pred)

      y_val_pred = net(x_val) 
      y_pred = torch.squeeze(y_pred)
      y_val_pred = torch.squeeze(y_val_pred)

      val_loss = criterion(y_val_pred, y_val)

      val_acc = calculate_accuracy(y_val, y_val_pred) 
      print("epoch {}\nTrain set - loss: {}, accuracy: {}\nTest  set - loss: {}, accuracy: {}"
            .format(epoch, 
                    round_tensor(train_loss), round_tensor(train_acc), 
                    round_tensor(val_loss), round_tensor(val_acc)))
    
    optimizer.zero_grad()
    
    train_loss.backward()
    
    optimizer.step()